In [1]:
import requests,csv,os
from bs4 import BeautifulSoup

# Task 1
# ---------
Scrap the USD To EGP Exchange rate from this website

https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html

and then use it to make a software that takes amount of USD Dollars from the user and calculate how much will it cost in EGP.


In [2]:
dollarToEG_PoundURL = 'https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html'
req = requests.get(dollarToEG_PoundURL)
htmlContent = BeautifulSoup(req.content,'html.parser')

In [3]:
divData = htmlContent.find('div',attrs={'class':"large-5 medium-5 small-12 columns nolpad"}).find_all('div',attrs={'class':"p_conv30"})[1]
print(divData.text.strip())
oneDollarToPound = float(divData.find('span').find('span').text)
print('EG Pound = '+str(oneDollarToPound*float(input('Enter amount of Dollar :'))))

1 USD = 31.0817 EGP
Enter amount of Dollar :4
EG Pound = 124.3268


In [4]:
import sys
import requests
from bs4 import BeautifulSoup
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLineEdit, QPushButton, QLabel

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout()

        self.text_input = QLineEdit(self)
        layout.addWidget(self.text_input)

        self.display_label = QLabel(self)
        layout.addWidget(self.display_label)

        self.btn = QPushButton('calculate', self)
        self.btn.clicked.connect(self.display_text)
        layout.addWidget(self.btn)

        self.setLayout(layout)
        self.setWindowTitle('PyQt Text Display')
        self.show()

    def coreFunction(self, UserNumber):
        dollarToEG_PoundURL = 'https://www.exchangerates.org.uk/Dollars-to-Egyptian-Pounds-currency-conversion-page.html'
        req = requests.get(dollarToEG_PoundURL)
        htmlContent = BeautifulSoup(req.content,'html.parser')
        divData = htmlContent.find('div',attrs={'class':"large-5 medium-5 small-12 columns nolpad"}).find_all('div',attrs={'class':"p_conv30"})[1]
        oneDollarToPound = float(divData.find('span').find('span').text)
        return str(oneDollarToPound*float(UserNumber))

    def display_text(self):
        text = self.text_input.text()
        self.display_label.setText(f'EG Pound = {self.coreFunction(text.strip())}')


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    sys.exit(app.exec_())


SystemExit: 0

C:\anaconda_IDE\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Task 2
# --------- 
Scraping from forecast weather
Imagine you work for forecast weather Now imagine your boss wants you to find the temperture for each day in celsius degree. How could you do this with Beautiful Soup?
https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168

In [5]:
dayData ,celsius,High = [],[],[]
counter = 1
def fahrenheit_to_celsius(temp):
    return (temp-32)/1.8
tempertureURL= 'https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168'
con = requests.get(tempertureURL)
tempData = BeautifulSoup(con.content,'html.parser')
ul = tempData.find('div',attrs={'id':'seven-day-forecast'}).find('div',attrs={'id':'seven-day-forecast-container'}).find('ul',attrs={'id':'seven-day-forecast-list'})
for highTemp in ul.find_all('p',attrs={'class':'period-name'}):
    dayData.append(highTemp.text.strip())
for lowTemp in ul.find_all('p',attrs={'class':'temp temp-low'}):
    celsius.append(fahrenheit_to_celsius(float(lowTemp.text.split(' ')[-2].strip())))
    celsius.append('')
for highTemp in ul.find_all('p',attrs={'class':'temp temp-high'}):
    High.append(fahrenheit_to_celsius(float(highTemp.text.split(' ')[-2].strip())))
for i in range(len(High)):
    celsius[i+counter]=High[i]
    counter+=1
    if i+1 == len(High):
        celsius.pop()
# print temperature in celsius:
for day,temp in zip(dayData,celsius):
    print(day,':',temp)

Tonight : 9.444444444444445
Tuesday : 16.11111111111111
TuesdayNight : 9.444444444444445
Wednesday : 15.0
WednesdayNight : 8.333333333333334
Thursday : 14.444444444444445
ThursdayNight : 8.333333333333334
Friday : 13.88888888888889
FridayNight : 8.88888888888889


# Task 3
# ---------
Scrap the books (name, price, rate) for each category and put them into a CSV & Excel file
https://books.toscrape.com/

In [6]:
from tqdm import tqdm
# productsURL= 'https://books.toscrape.com/'

for page in tqdm(range(1,51)):
    productsURL = 'https://books.toscrape.com/catalogue/page-'+str(page)+'.html'
    requestProduct=requests.get(productsURL)
    soupPage = BeautifulSoup(requestProduct.content,'html.parser')
    ol = soupPage.find('div',attrs={'class':'col-sm-8 col-md-9'}).find('section').find_all('div')[1].find('ol',attrs={'class':'row'})
    mode = 'a' if os.path.exists('products.csv') else 'w'
    with open('products.csv',mode = mode,newline='',encoding='utf-8') as f:
        w = csv.writer(f)
        if mode == 'w':
            w.writerow(['Book Name','Book Price','Book Rate'])
        for li in ol.find_all('li',attrs={'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'}):
            bookName = li.find('h3').find('a').attrs['title'].strip()
            bookPrice = li.find('div',attrs={'class':'product_price'}).find('p',attrs={'class':'price_color'}).text
            bookRate = li.find('p').attrs['class'][-1]
            w.writerow([bookName,bookPrice,bookRate])

100%|███████████████████████████████████████████████████████████| 50/50 [00:33<00:00,  1.50it/s]


In [7]:
!products.csv

# Task 4
# ---------
once you decided to do some exploring an want to know each country's (name, capital, population, Area(km^2) 
scrape the 250 countries (name, capital, population, area) and save them into a csv file
use this site to scrape this data : https://www.scrapethissite.com/pages/simple/

In [8]:
countriesURL= 'https://www.scrapethissite.com/pages/simple/'
countriesReq = requests.get(countriesURL)
countriespage=BeautifulSoup(countriesReq.content,'html.parser')
countriesData=countriespage.find('div',attrs={'id':'page'}).find('div',attrs={'class':'container'}).find_all('div',attrs={'class':'col-md-4 country'})
mode = 'a' if os.path.exists('countriesData.csv') else 'w'
with open('countriesData.csv',mode = mode ,newline='',encoding='utf-8') as f:
    w = csv.writer(f)
    if mode == 'w':
        w.writerow(['Country Name','Capital','Population','Area (km2)'])
    for cnt in countriesData:
        countryName=cnt.find('h3').text.strip()
        countryCapital=cnt.find('span',attrs={'class':'country-capital'}).text.strip()
        countryPopulation=cnt.find('span',attrs={'class':'country-population'}).text.strip()
        countriesArea=cnt.find('span',attrs={'class':'country-area'}).text.strip()
        w.writerow([countryName,countryCapital,countryPopulation,countriesArea])

In [9]:
import pandas as pd
pd.read_csv('countriesData.csv')

,Country Name,Capital,Population,Area (km2)
0,Andorra,Andorra la Vella,84000,468.0
1,United Arab Emirates,Abu Dhabi,4975593,82880.0
2,Afghanistan,Kabul,29121286,647500.0
3,Antigua and Barbuda,St. John's,86754,443.0
4,Anguilla,The Valley,13254,102.0
...,...,...,...,...
245,Yemen,Sanaa,23495361,527970.0
246,Mayotte,Mamoudzou,159042,374.0
247,South Africa,Pretoria,49000000,1219912.0
248,Zambia,Lusaka,13460305,752614.0


# Task 5
# ---------
scraping data about hockey teams:
you have to scrap the first page table's data (Team name, year, wins, losses, win%, goals for(gf), golaagainst(ga)
from this website: https://www.scrapethissite.com/pages/forms/?page_num=1

In [10]:
from tqdm import tqdm
tableHeader=[]
tableBody=[]
hockeyTeamsURL = 'https://www.scrapethissite.com/pages/forms/?page_num='
mode = 'a' if os.path.exists('hockeyTeam.csv') else 'w'
for i in tqdm(range(1,25)):
    hockeyReq = requests.get(hockeyTeamsURL+str(i))
    hockeyContent = BeautifulSoup(hockeyReq.content,'html.parser')
    mode = 'a' if os.path.exists('hockeyTeam.csv') else 'w'
    with open('hockeyTeam.csv',mode=mode,newline='') as f:
        w = csv.writer(f)
        # Header
        if mode == 'w':
            for header in hockeyContent.find('table').find_all('th'):
                tableHeader.append(header.text.strip())
            w.writerow(tableHeader)
        # Body
        for row in hockeyContent.find('table').find_all('tr')[1:]:
            for cell in row.find_all('td'):
                tableBody.append(cell.text.strip())
            w.writerow(tableBody)
            tableBody.clear()

100%|███████████████████████████████████████████████████████████| 24/24 [00:16<00:00,  1.49it/s]


In [11]:
!hockeyTeam.csv

# Task 6
# ---------
Use this web service 
https://jsonplaceholder.typicode.com/users
and make a program that get all users data including name, username, email, street, suite, city, zip code, geo_lat & geo_long and then save them in a CSV & Excel file.

In [12]:
linkUsersData = 'https://jsonplaceholder.typicode.com/users'
userData=requests.get(linkUsersData).json()

mode = 'a' if os.path.exists('UsersData.csv') else 'w'
with open('UsersData.csv',mode=mode,newline='',encoding='utf-8') as f:
    w = csv.writer(f)
    w.writerow(['id','name','username','email','address street','address suite','address city','address zipcode','address geo','phone','website','company name','company catchPhrase','company bs'])
    for user in userData:
        w.writerow([user['id'],user['name'],user['username'],user['email'],user['address']['street'],user['address']['suite'],user['address']['city'],user['address']['zipcode'],user['address']['geo'],user['phone'],user['website'],user['company']['name'],user['company']['catchPhrase'],user['company']['bs']])

In [13]:
import pandas as pd
pd.read_csv('UsersData.csv')

,id,name,username,email,address street,address suite,address city,address zipcode,address geo,phone,website,company name,company catchPhrase,company bs
0,1,Leanne Graham,Bret,Sincere@april.biz,Kulas Light,Apt. 556,Gwenborough,92998-3874,"{'lat': '-37.3159', 'lng': '81.1496'}",1-770-736-8031 x56442,hildegard.org,Romaguera-Crona,Multi-layered client-server neural-net,harness real-time e-markets
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,Victor Plains,Suite 879,Wisokyburgh,90566-7771,"{'lat': '-43.9509', 'lng': '-34.4618'}",010-692-6593 x09125,anastasia.net,Deckow-Crist,Proactive didactic contingency,synergize scalable supply-chains
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,Douglas Extension,Suite 847,McKenziehaven,59590-4157,"{'lat': '-68.6102', 'lng': '-47.0653'}",1-463-123-4447,ramiro.info,Romaguera-Jacobson,Face to face bifurcated interface,e-enable strategic applications
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,Hoeger Mall,Apt. 692,South Elvis,53919-4257,"{'lat': '29.4572', 'lng': '-164.2990'}",493-170-9623 x156,kale.biz,Robel-Corkery,Multi-tiered zero tolerance productivity,transition cutting-edge web services
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,Skiles Walks,Suite 351,Roscoeview,33263,"{'lat': '-31.8129', 'lng': '62.5342'}",(254)954-1289,demarco.info,Keebler LLC,User-centric fault-tolerant solution,revolutionize end-to-end systems
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,Norberto Crossing,Apt. 950,South Christy,23505-1337,"{'lat': '-71.4197', 'lng': '71.7478'}",1-477-935-8478 x6430,ola.org,Considine-Lockman,Synchronised bottom-line interface,e-enable innovative applications
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,Rex Trail,Suite 280,Howemouth,58804-1099,"{'lat': '24.8918', 'lng': '21.8984'}",210.067.6132,elvis.io,Johns Group,Configurable multimedia task-force,generate enterprise e-tailers
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,Ellsworth Summit,Suite 729,Aliyaview,45169,"{'lat': '-14.3990', 'lng': '-120.7677'}",586.493.6943 x140,jacynthe.com,Abernathy Group,Implemented secondary concept,e-enable extensible e-tailers
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,Dayna Park,Suite 449,Bartholomebury,76495-3109,"{'lat': '24.6463', 'lng': '-168.8889'}",(775)976-6794 x41206,conrad.com,Yost and Sons,Switchable contextually-based project,aggregate real-time technologies
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,Kattie Turnpike,Suite 198,Lebsackbury,31428-2261,"{'lat': '-38.2386', 'lng': '57.2232'}",024-648-3804,ambrose.net,Hoeger LLC,Centralized empowering task-force,target end-to-end models
